## Notebook for cleaning the GSOD data

Author: Wenwen Kong

Updated: 2022-07-16

### About the data
- The GSOD data was downloaded using the R package [GSODR](https://cran.r-project.org/web/packages/GSODR/vignettes/GSODR.html), see [here](https://github.com/wenwenkong/random_collection_r/blob/main/download_GSOD_by_station.R) for an example download script

- Time span: 1973-2021

- Frequency: Daily 

- Downloaded raw data:

    - 722950-23174_KLAX_1973-2021.csv
    - 722886-23130_KVNY_1973-2021.csv
    - 722880-23152_KBUR_1973-2021.csv
    - 722885-99999_KSMO_1973-2005.csv
    - 722885-93197_KSMO_2006-2021.csv
        
- Processed data:

    - GSOD_KLAX_1973-2021_cleaned.csv
    - GSOD_KVNY_1973-2021_cleaned.csv
    - GSOD_KBUR_1973-2021_cleaned.csv
    - GSOD_KSMO_1973-2021_cleaned.csv

### Purpose of this notebook

- Extract variables of interest
- Remove leap days
- Remove duplicated days
- Remove years when 
    - There are >=20% days missing from that year
    - There are >=20% days missing from that summer
- Save the cleaned data 

### Disclaimer:
The method used in this notebook might not be the most efficient way to deal with GSOD. 

### Load useful libraries 

In [1]:
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt

---
### To print out missing years from data, and to remove years that contains too many missing days

We save the cleaned data from this step as temporary .csv files. 

In [2]:
file_names    = ['722880-23152_KBUR_1973-2021.csv', 
                 '722886-23130_KVNY_1973-2021.csv',
                 '722950-23174_KLAX_1973-2021.csv']
station       = ['KBUR', 
                 'KVNY',
                 'KLAX']

years         = list(range(1973, 2022))
days_of_year  = 365
days_of_summer= 92

for file, site in zip(file_names, station):
    print("*********************************************************")
    print("******* station = "+site+"  ***********")
    print("*********************************************************")
    
    # ------ all_dfs
    all_dfs = []
    
    # ------ read in data
    df = pd.read_csv(file)
    
    # ------ remove leap days
    df = df[~((df.MONTH == 2) & (df.DAY == 29))]

    # ------ remove duplicated days if there are any
    df.drop_duplicates(subset='YEARMODA', keep = False, inplace = True)
    
    # *********************************************************************************
    #    To deal with missing dates, we want to 
    #    (1) either remove rows of years that contains too many missing values
    #    (2) or set missing days with NaN if there are only a few days missed 
    #    
    #    For (1), it's best to remove the rows based on the row label, here we can ues 'YEARMODA' 
    #    For label-based deletion, we set the index first on the dataframe
    # *********************************************************************************
    
    df           = df.set_index('YEARMODA') 
    df.index     = pd.to_datetime(df.index)
    
    for year in years:
        
        df_year  = df[df.YEAR == year]
        df_summer= df[(df.YEAR == year) & (df.MONTH >= 6) & (df.MONTH <=8)]
        
        # *********************************************************
        #    Deal with the missing dates
        # *********************************************************
        if len(df_year.DAY) == 0:
            print("--------------------------------------------------------------------")
            print('The whole year of '+str(year)+' is missing')
            
        elif len(df_year.DAY) < 0.8 * days_of_year:
            print("--------------------------------------------------------------------")
            print("Abandon year "+str(year)+" because it has missed more than 20% days")
            df[df.YEAR == year] = np.nan 

            
        elif len(df_summer.DAY) < 0.8 * days_of_summer:
            print("--------------------------------------------------------------------")
            print("Abandon year "+str(year)+" because it has missed more than 20% days during summer")
            df[df.YEAR == year] = np.nan
    
    df.dropna()
    df.to_csv('./GSOD_'+site+'_1973-2021_temporary.csv')

*********************************************************
******* station = KBUR  ***********
*********************************************************


/Users/wenwen/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (32,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


--------------------------------------------------------------------
The whole year of 2000 is missing
--------------------------------------------------------------------
The whole year of 2001 is missing
--------------------------------------------------------------------
The whole year of 2002 is missing
--------------------------------------------------------------------
The whole year of 2003 is missing
*********************************************************
******* station = KVNY  ***********
*********************************************************
--------------------------------------------------------------------
Abandon year 1973 because it has missed more than 20% days
--------------------------------------------------------------------
Abandon year 1987 because it has missed more than 20% days
--------------------------------------------------------------------
The whole year of 2000 is missing
--------------------------------------------------------------------
The whol

--- 
### We now deal with years that contain a small portion of missing days 

We read in the temporary .csv from the previous step, and insert missing days with NaN.

In [3]:
station       = ['KBUR', 
                 'KVNY',
                 'KLAX']

years         = list(range(1973, 2022))
days_of_year  = 365
days_of_summer= 92

for site in station:
    print("*********************************************************")
    print("******* station = "+site+"  ***********")
    print("*********************************************************")
    
    all_dfs = []
    
    df = pd.read_csv('./GSOD_'+site+'_1973-2021_temporary.csv')
    
    df           = df.set_index('YEARMODA') 
    df.index     = pd.to_datetime(df.index)
    
    for year in years:
        df_year   = df[df.YEAR == year]
        df_summer = df[(df.YEAR == year) & (df.MONTH >= 6) & (df.MONTH <=8)]
        
        if len(df_year.DAY) < days_of_year and len(df_year.DAY) >= 0.8 * days_of_year:
            print("--------------------------------------------------------------------")
            missed = days_of_year - len(df_year.DAY)
            print("Year "+str(year)+" has "+str(missed)+" missing days")

            date_range        = pd.date_range(start=str(year)+"-01-01", end=str(year)+"-12-31")
            print(date_range.difference(df_year.index))
        
            # ---- fill in the missing dates with a missing value
            df_year           = df_year.reindex(date_range)
        
        all_dfs.append(df_year)
        
    all_dfs_save = pd.concat(all_dfs)
    all_dfs_save.to_csv('./GSOD_'+site+'_1973-2021_cleaned.csv')

*********************************************************
******* station = KBUR  ***********
*********************************************************
--------------------------------------------------------------------
Year 1975 has 4 missing days
DatetimeIndex(['1975-05-31', '1975-07-01', '1975-07-02', '1975-07-03'], dtype='datetime64[ns]', freq=None)
--------------------------------------------------------------------
Year 1976 has 2 missing days
DatetimeIndex(['1976-02-29', '1976-07-27', '1976-07-28'], dtype='datetime64[ns]', freq=None)
--------------------------------------------------------------------
Year 1978 has 1 missing days
DatetimeIndex(['1978-12-03'], dtype='datetime64[ns]', freq=None)
--------------------------------------------------------------------
Year 1982 has 1 missing days
DatetimeIndex(['1982-02-21'], dtype='datetime64[ns]', freq=None)
--------------------------------------------------------------------
Year 1986 has 1 missing days
DatetimeIndex(['1986-04-21']

--- 

### KSMO is a bit different 

Because KSMO data is contained in two separate files, which might be due to the station ID change at a certain point. Anyway, we need to first merge the two KSMO csv files to a combined file, then clean the data. 

In [4]:
files = ['722885-99999_KSMO_1973-2005.csv','722885-93197_KSMO_2006-2021.csv']

KSMO_df = []

for f in files:
    
    df  = pd.read_csv(f)
    
    KSMO_df.append(df)

KSMO    = pd.concat(KSMO_df)
KSMO.to_csv('./GSOD_KSMO_1973-2021_combined.csv')

In [5]:
df = pd.read_csv('./GSOD_KSMO_1973-2021_combined.csv')

# ------ remove leap days
df = df[~((df.MONTH == 2) & (df.DAY == 29))]

# ------ remove duplicated days if there are any
df.drop_duplicates(subset='YEARMODA', keep = False, inplace = True)
    
    # *********************************************************************************
    #    To deal with missing dates, we want to 
    #    (1) either remove rows of years that contains too many missing values
    #    (2) or set missing days with NaN if there are only a few days missed 
    #    
    #    For (1), it's best to remove the rows based on the row label, here we can ues 'YEARMODA' 
    #    For label-based deletion, we set the index first on the dataframe
    # *********************************************************************************
    
df           = df.set_index('YEARMODA') 
df.index     = pd.to_datetime(df.index)
    
for year in years:
        
    df_year  = df[df.YEAR == year]
    df_summer= df[(df.YEAR == year) & (df.MONTH >= 6) & (df.MONTH <=8)]
        
    # *********************************************************
    #    Deal with the missing dates
    # *********************************************************
    if len(df_year.DAY) == 0:
        print("--------------------------------------------------------------------")
        print('The whole year of '+str(year)+' is missing')
            
    elif len(df_year.DAY) < 0.8 * days_of_year:
        print("--------------------------------------------------------------------")
        print("Abandon year "+str(year)+" because it has missed more than 20% days")
            #date_range          = pd.date_range(start=str(year)+"-01-01", end=str(year)+"-12-31")
            #df = df.drop([str(year)+'-01-01': str(year)+'-12-31'], axis = 0)
        df[df.YEAR == year] = np.nan 
            #df = df.drop(str(year), axis = 0)
            #df  = df[df.YEAR == year].drop(axis=0)
                        
    elif len(df_summer.DAY) < 0.8 * days_of_summer:
        print("--------------------------------------------------------------------")
        print("Abandon year "+str(year)+" because it has missed more than 20% days during summer")
            #df = df.drop(df.YEAR == year)
        df[df.YEAR == year] = np.nan
            #date_range          = pd.date_range(start=str(year)+"-01-01", end=str(year)+"-12-31")
            #df = df.drop(date_range, axis = 0)
            #df = df.drop(str(year), axis = 0)
            #df  = df[df.YEAR == year].drop(axis=0)
    
df.dropna()
df.to_csv('./GSOD_KSMO_1973-2021_temporary.csv')
    

--------------------------------------------------------------------
The whole year of 1973 is missing
--------------------------------------------------------------------
The whole year of 1974 is missing
--------------------------------------------------------------------
Abandon year 1975 because it has missed more than 20% days
--------------------------------------------------------------------
The whole year of 1976 is missing
--------------------------------------------------------------------
The whole year of 1977 is missing
--------------------------------------------------------------------
The whole year of 1978 is missing
--------------------------------------------------------------------
The whole year of 1979 is missing
--------------------------------------------------------------------
The whole year of 1980 is missing
--------------------------------------------------------------------
The whole year of 1981 is missing
------------------------------------------------

In [6]:
all_dfs = []
    
df = pd.read_csv('./GSOD_KSMO_1973-2021_combined.csv')
    
df           = df.set_index('YEARMODA') 
df.index     = pd.to_datetime(df.index)
    
for year in years:
    df_year   = df[df.YEAR == year]
    df_summer = df[(df.YEAR == year) & (df.MONTH >= 6) & (df.MONTH <=8)]
        
    if len(df_year.DAY) < days_of_year and len(df_year.DAY) >= 0.8 * days_of_year:
        print("--------------------------------------------------------------------")
        missed = days_of_year - len(df_year.DAY)
        print("Year "+str(year)+" has "+str(missed)+" missing days")

        date_range        = pd.date_range(start=str(year)+"-01-01", end=str(year)+"-12-31")
        print(date_range.difference(df_year.index))
        
        # ---- fill in the missing dates with a missing value
        df_year           = df_year.reindex(date_range)
        
    all_dfs.append(df_year)
        
all_dfs_save = pd.concat(all_dfs)
all_dfs_save.to_csv('./GSOD_KSMO_1973-2021_cleaned.csv')

--------------------------------------------------------------------
Year 1983 has 28 missing days
DatetimeIndex(['1983-01-29', '1983-01-30', '1983-01-31', '1983-02-01',
               '1983-02-02', '1983-02-03', '1983-02-04', '1983-02-05',
               '1983-02-06', '1983-02-07', '1983-02-08', '1983-09-29',
               '1983-09-30', '1983-12-17', '1983-12-18', '1983-12-19',
               '1983-12-20', '1983-12-21', '1983-12-22', '1983-12-23',
               '1983-12-24', '1983-12-25', '1983-12-26', '1983-12-27',
               '1983-12-28', '1983-12-29', '1983-12-30', '1983-12-31'],
              dtype='datetime64[ns]', freq=None)
--------------------------------------------------------------------
Year 1984 has 22 missing days
DatetimeIndex(['1984-01-01', '1984-01-02', '1984-01-03', '1984-01-04',
               '1984-01-05', '1984-01-06', '1984-01-07', '1984-01-08',
               '1984-01-09', '1984-01-10', '1984-01-11', '1984-01-12',
               '1984-01-13', '1984-01-14',